# Dadbot: Dad's memorial bot based on RASA (old style)

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


# Installations
* Rasa
* SpaCy Language Model

In [2]:
import sys
python = sys.executable

# In your environment run:
#!{python} -m pip install -U pip
#!{python} -m pip install tensorflow[tensorflow-addons]==2.1.0
#!{python} -m pip install rasa[convert]~=1.10.0

In [3]:
!{python} -m spacy download es_core_news_md

You should consider upgrading via the '/home/debian/.conda/envs/rasa/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


## Downloading the Spanish Language Model

In [4]:
!{python} -m spacy link es_core_news_md es --force;

✔ Linking successful
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/es_core_news_md -->
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


## Import the Installations

In [5]:
import rasa


# 1. Teaching the bot to understand user inputs using Rasa NLU

## Training the NLU Model.

In [6]:
from rasa.model import get_model
from rasa.nlu import config, training_data, utils
from rasa.nlu.utils import write_to_file
from rasa.nlu.components import ComponentBuilder
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Interpreter, Trainer, TrainingData
from rasa.nlu.components import Component
from rasa.nlu.tokenizers.tokenizer import Token
from rasa.utils.tensorflow.constants import ENTITY_RECOGNITION

import spacy

#spacy_parser = spacy.load('es_core_news_md')
#nlp = spacy.load('es')

# loading the nlu training samples
training_data = training_data.loading.load_data("data/nlu/nlu-papaito.md")

# trainer to train our pipeline
trainer = Trainer(config.load("config_simple.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa.nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component RegexFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EmbeddingIntentClassifier
Epochs: 100%|██████████| 300/300 [00:26<00:00, 11.24it/s, t_loss=1.192, i_loss=0.188, i_acc=0.989] 
INFO:rasa.utils.tensorflow.models:Finished training.
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:S

## Evaluating the NLU model on a random text

In [7]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("dejándome el coche"))

{
  "intent": {
    "name": "ask_help",
    "confidence": 0.9746991395950317
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "ask_help",
      "confidence": 0.9746991395950317
    },
    {
      "name": "ask_cuenta_algo",
      "confidence": 0.00730551453307271
    },
    {
      "name": "goodbye",
      "confidence": 0.005326487123966217
    },
    {
      "name": "mood_happy",
      "confidence": 0.004319251049309969
    },
    {
      "name": "ask_pais",
      "confidence": 0.0017294354038313031
    },
    {
      "name": "mood_unhappy",
      "confidence": 0.0016026819357648492
    },
    {
      "name": "mood_affirm",
      "confidence": 0.0014282844495028257
    },
    {
      "name": "ask_ocupacion",
      "confidence": 0.0013754114042967558
    },
    {
      "name": "ask_ultimas_noticias",
      "confidence": 0.001346154953353107
    },
    {
      "name": "ask_razon",
      "confidence": 0.0008676530560478568
    }
  ],
  "text": "dej\u00e1ndome el coche"
}


## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [8]:
from rasa.nlu.test import run_evaluation

run_evaluation("data/nlu/nlu-papaito.md", model_directory)

INFO:rasa.nlu.test:Running model for predictions:
100%|██████████| 612/612 [00:01<00:00, 387.19it/s]
INFO:rasa.nlu.test:Intent evaluation results:
INFO:rasa.nlu.test:Intent Evaluation: Only considering those 612 examples that have a defined intent out of 612 examples
INFO:rasa.nlu.test:F1-Score:  0.9885768159079111
INFO:rasa.nlu.test:Precision: 0.9887989278108248
INFO:rasa.nlu.test:Accuracy:  0.988562091503268
INFO:rasa.nlu.test:Classification report: 
                      precision    recall  f1-score   support

     inform_pregunta       1.00      1.00      1.00        23
        ask_vivienda       1.00      1.00      1.00        13
           ask_razon       1.00      1.00      1.00        12
        mood_unhappy       0.98      1.00      0.99        87
          ask_comida       1.00      0.96      0.98        25
    ask_deporte_bici       1.00      1.00      1.00         9
          ask_gustos       1.00      1.00      1.00         4
  ask_deporte_futbol       1.00      1.00     

{'intent_evaluation': {'predictions': [{'text': '¡ey!',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9902545809745789},
   {'text': '¡eyyy!',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9927659630775452},
   {'text': 'hola tú',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9950994849205017},
   {'text': 'hola',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.995073676109314},
   {'text': 'hola de nuevo',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9960068464279175},
   {'text': 'buenos días',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9909021258354187},
   {'text': 'buenos días por la mañana',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9904657006263733},
   {'text': 'buenas tardes',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.5369973182678223},
   {'text': 'buenas tardes por las tardes',
    

# 2. Teaching the bot to respond using Rasa Core

##  Visualising the Training Data

In [9]:
#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!breq install graphviz

#!conda install -y -n rasa pygraphviz pkg-config;

In [10]:
from IPython.display import Image
from rasa.core.agent import Agent
 
agent = Agent('domain-papaito.yml')
#agent.visualize("data/core/stories-papaito.md", "story_graph.png", max_history=2)
#Image(filename="story_graph.png")

## Training a Dialogue Model

In [16]:
from rasa.core.policies.policy import Policy
from rasa.core.policies.registry import FallbackPolicy, KerasPolicy, MemoizationPolicy, FormPolicy, MappingPolicy
from rasa.core.agent import Agent

import asyncio

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain-papaito.yml',
              policies=[MappingPolicy(),
                        MemoizationPolicy(max_history=5),
                        KerasPolicy(validation_split=0.1,epochs=630),
                        FormPolicy(),
                        fallback])

# loading our neatly defined training dialogues
training_data = await agent.load_data('data/core/stories-papaito.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed trackers: 100%|██████████| 28/28 [00:00<00:00, 595.88it/s, # actions=76]
Processed actions: 76it [00:00, 2205.28it/s, # examples=72]
Processed trackers:  47%|████▋     | 250/528 [00:00<00:00, 293.77it/s, # actions=925]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1473/1473 [==============================] - 2s 1ms/sample - loss: 3.1708 - accuracy: 0.4121
Epoch 2/630
1473/1473 [==============================] - 0s 79us/sample - loss: 2.4209 - accuracy: 0.4569
Epoch 3/630
1473/1473 [==============================] - 0s 80us/sample - loss: 2.2812 - accuracy: 0.4569
Epoch 4/630
1473/1473 [==============================] - 0s 77us/sample - loss: 2.1923 - accuracy: 0.4569
Epoch 5/630
1473/1473 [==============================] - 0s 77us/sample - loss: 2.1210 - accuracy: 0.4569
Epoch 6/630
1473/1473 [==============================] - 0s 78us/sample - loss: 2.0398 - accuracy: 0.4569
Epoch 7/630
1473/1473 [==============================] - 0s 78us/sample - loss: 1.9516 - accuracy: 0.4569
Epoch 8/630
1473/1473 [==============================] - 0s 77us/sample - loss: 1.8513 - accuracy: 0.4637
Epoch 9/630
1473/1473 [==============================] - 0s 77us/sample - loss: 1.7233 - accuracy: 0.4997
Epoch 10/630
1473/1473 [==============================] - 0

1473/1473 [==============================] - 0s 78us/sample - loss: 0.2599 - accuracy: 0.9179
Epoch 78/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.2399 - accuracy: 0.9212
Epoch 79/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.2383 - accuracy: 0.9192
Epoch 80/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.2379 - accuracy: 0.9206
Epoch 81/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.2100 - accuracy: 0.9294
Epoch 82/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.2462 - accuracy: 0.9212
Epoch 83/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.2846 - accuracy: 0.9097
Epoch 84/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.2418 - accuracy: 0.9165
Epoch 85/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.2471 - accuracy: 0.9192
Epoch 86/630
1473/1473 [==========================

1473/1473 [==============================] - 0s 83us/sample - loss: 0.2570 - accuracy: 0.9151
Epoch 154/630
1473/1473 [==============================] - 0s 82us/sample - loss: 0.2211 - accuracy: 0.9185
Epoch 155/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.2174 - accuracy: 0.9280
Epoch 156/630
1473/1473 [==============================] - 0s 81us/sample - loss: 0.2052 - accuracy: 0.9321
Epoch 157/630
1473/1473 [==============================] - 0s 81us/sample - loss: 0.2182 - accuracy: 0.9212
Epoch 158/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.2272 - accuracy: 0.9226
Epoch 159/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.2022 - accuracy: 0.9294
Epoch 160/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.2325 - accuracy: 0.9179
Epoch 161/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.2016 - accuracy: 0.9341
Epoch 162/630
1473/1473 [=================

Epoch 229/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.2330 - accuracy: 0.9165
Epoch 230/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.2158 - accuracy: 0.9212
Epoch 231/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.2016 - accuracy: 0.9253
Epoch 232/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1885 - accuracy: 0.9348
Epoch 233/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.2001 - accuracy: 0.9314
Epoch 234/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1837 - accuracy: 0.9348
Epoch 235/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.2041 - accuracy: 0.9274
Epoch 236/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.2150 - accuracy: 0.9240
Epoch 237/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.1790 - accuracy: 0.9375
Epoch 238/630
1473/1473 [===

...

Epoch 380/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.2225 - accuracy: 0.9185
Epoch 381/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.1733 - accuracy: 0.9430
Epoch 382/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.1628 - accuracy: 0.9389
Epoch 383/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1718 - accuracy: 0.9375
Epoch 384/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1989 - accuracy: 0.9233
Epoch 385/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.1822 - accuracy: 0.9341
Epoch 386/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.1953 - accuracy: 0.9287
Epoch 387/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.1807 - accuracy: 0.9308
Epoch 388/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.1801 - accuracy: 0.9287
Epoch 389/630
1473/1473 [===

1473/1473 [==============================] - 0s 81us/sample - loss: 0.1859 - accuracy: 0.9274
Epoch 456/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1911 - accuracy: 0.9294
Epoch 457/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.1958 - accuracy: 0.9321
Epoch 458/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1688 - accuracy: 0.9375
Epoch 459/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.1880 - accuracy: 0.9301
Epoch 460/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1710 - accuracy: 0.9369
Epoch 461/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.2022 - accuracy: 0.9328
Epoch 462/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1926 - accuracy: 0.9335
Epoch 463/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1791 - accuracy: 0.9341
Epoch 464/630
1473/1473 [=================

Epoch 531/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.1636 - accuracy: 0.9375
Epoch 532/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.1893 - accuracy: 0.9321
Epoch 533/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1659 - accuracy: 0.9348
Epoch 534/630
1473/1473 [==============================] - 0s 78us/sample - loss: 0.1730 - accuracy: 0.9355
Epoch 535/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.1862 - accuracy: 0.9246
Epoch 536/630
1473/1473 [==============================] - 0s 81us/sample - loss: 0.1921 - accuracy: 0.9280
Epoch 537/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.2023 - accuracy: 0.9192
Epoch 538/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1798 - accuracy: 0.9301
Epoch 539/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.1760 - accuracy: 0.9348
Epoch 540/630
1473/1473 [===

1473/1473 [==============================] - 0s 79us/sample - loss: 0.1607 - accuracy: 0.9382
Epoch 607/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1623 - accuracy: 0.9375
Epoch 608/630
1473/1473 [==============================] - 0s 80us/sample - loss: 0.1810 - accuracy: 0.9314
Epoch 609/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1980 - accuracy: 0.9308
Epoch 610/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1863 - accuracy: 0.9328
Epoch 611/630
1473/1473 [==============================] - 0s 77us/sample - loss: 0.1893 - accuracy: 0.9294
Epoch 612/630
1473/1473 [==============================] - 0s 81us/sample - loss: 0.1825 - accuracy: 0.9267
Epoch 613/630
1473/1473 [==============================] - 0s 81us/sample - loss: 0.1494 - accuracy: 0.9389
Epoch 614/630
1473/1473 [==============================] - 0s 79us/sample - loss: 0.1604 - accuracy: 0.9382
Epoch 615/630
1473/1473 [=================

Processed trackers: 100%|██████████| 28/28 [00:00<00:00, 639.30it/s, # actions=73]
INFO:rasa.core.agent:Model directory models/dialogue/core exists and contains old model files. All files will be overwritten.
INFO:rasa.core.agent:Persisted model to '/home/debian/workspace/Dadbot/models/dialogue/core'


# Talk to your Bot

In [17]:
#Starting the Bot
from rasa.core.agent import Agent
from rasa.core.utils import EndpointConfig

action_endpoint = EndpointConfig(url="http://0.0.0.0:5055/webhook")
agent = Agent.load('models/dialogue', interpreter=model_directory, action_endpoint=action_endpoint)

#!docker run -d -p 5055:5055 --mount type=bind,source=/home/debian/workspace/Dadbot/actions/actions.py,target=/app/actions rasa/rasa-sdk

## Creating the voice synthesizer

In [13]:
#!git clone https://github.com/NVIDIA/tacotron2.git

from tacotron2.hparams import create_hparams
from tacotron2.model import Tacotron2
from tacotron2.layers import TacotronSTFT, STFT
from tacotron2.audio_processing import griffin_lim
from tacotron2.train import load_model
from tacotron2.text import text_to_sequence
from tacotron2.waveglow.mel2samp import files_to_list, MAX_WAV_VALUE
from tacotron2.denoiser import Denoiser
import numpy as np
import torch

def synthesize(text, voice, sigma=0.6, denoiser_strength=0.05, is_fp16=False):

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    if voice == "papaito":
        voice_model = "nvidia_tacotron2_papaito_300"
    elif voice == "constantino":
        voice_model = "tacotron2_Constantino_600"
    elif voice == "orador":
        voice_model = "checkpoint_tacotron2_29000_es"
    
    checkpoint_path = "/home/debian/workspace/models/" + voice_model
        
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    waveglow_path = '/home/debian/workspace/models/waveglow_256channels_ljs_v2.pt'
    waveglow = torch.load(waveglow_path)['model']
    _ = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)

    #text="¡Cágate lorito!"
    #with open(filelist_path, encoding='utf-8', mode='r') as f:
    #    text = f.read()

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    #mel = torch.unsqueeze(mel, 0)
    mel = mel_outputs.half() if is_fp16 else mel_outputs
    audio = np.array([])
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=sigma)
        if denoiser_strength > 0:
             audio = denoiser(audio, denoiser_strength)
        audio = audio * MAX_WAV_VALUE
        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
    
    return audio, hparams.sampling_rate

## Start chat

In [ ]:
import sounddevice as sd
from sty import fg, bg, ef, rs

print("¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'")
while True:
    a = input()
    
    if a == 'quieto parao':
        break
    responses = await agent.handle_text(a)
    for response in responses:
        to_synth = response["text"]
        #to_synth = "Esto es una prueba para ver si funciona"
        print(fg.blue + to_synth + fg.rs)
        response_file = open('response.txt','w') 
        response_file.write(to_synth)
        #voice, sr = synthesize(to_synth, "orador")
        #sd.play(voice, sr)
        response_file.close()        

¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'
dime las noticias


ERROR:rasa.core.processor:Encountered an exception while running action 'action_ultimas_noticias'. Bot will continue, but the actions events are lost. Please check the logs of your action server for more information.


dime las noticias
Estas son las últimas noticias: Rafa Nadal fue líder hasta el hoyo diez, acaba cuarto.
